# Clean Google Mobility Data 



In [ ]:
# Dependencies and Setup
import json
import os
import pandas as pd
import urllib.request
import requests
# from config import db_pwd, db_user
from sqlalchemy import create_engine

## Store Google CSV into DataFrame

In [ ]:
csv_file = "Resources/google_mob_US.csv"
google_data_df = pd.read_csv(csv_file)
google_data_df.head()

### Rename the dataframe with select columns

In [ ]:
google_data_df = google_data_df.rename(columns = {"State":'states', "date":'dates', "retail_and_recreation":'retail_recreation',
                            "grocery_and_pharmacy":'grocery_pharmacy', "parks":'parks', "transit_stations":"transit", "workplaces":"workplaces", "residential":"residential"})
google_data_df.head()

In [ ]:
google_df = google_data_df[["states", "dates", "retail_recreation",
                            "grocery_pharmacy", "parks", "transit", "workplaces", "residential"]]
google_df.head()

In [ ]:
# Reseting the index and saving the cleaned file to csv
# clean_google_mob_US_df = clean_google_mob_US_df.reset_index(drop = True)
# clean_google_mob_US_df.to_csv("../Data/clean_google_mob_US.csv")

In [ ]:
google_df["SMA_retail_recreation"] = google_df.iloc[:,2].rolling(window=30).mean()
google_df["SMA_grocery_pharmacy"] = google_df.iloc[:,3].rolling(window=30).mean()
google_df["SMA_parks"] = google_df.iloc[:,4].rolling(window=30).mean()
google_df["SMA_transit"] = google_df.iloc[:,5].rolling(window=30).mean()
google_df["SMA_workplaces"] = google_df.iloc[:,6].rolling(window=30).mean()
google_df["SMA_residential"] = google_df.iloc[:,7].rolling(window=30).mean()
google_df.head()


In [ ]:
# Reseting the index and saving the cleaned file to csv
google_us_df = google_df.reset_index(drop = True)
google_us_df.to_csv("google_us.csv")


In [ ]:
# Selecting only the data for the US. This dropped the data to 456634 rows × 14 columns
google_mob_VA = google_df.loc[google_df["states"] == "Virginia"]
google_mob_VA = google_mob_VA.reset_index(drop = True)
google_mob_VA.head()

In [ ]:
# Grouping by date, so we can get all the data for all states into one date
# skipnabool, default is True, and all NA/null values are excluded, when computing the result.
data_by_date_VA_df = pd.DataFrame(google_mob_VA.groupby("dates").mean())
data_by_date_VA_df.reset_index(inplace = True)
data_by_date_VA_df.head()

In [ ]:
# Reseting the index and saving the cleaned file to csv
data_by_date_VA_df.to_csv("google_mob_VAA.csv")

In [ ]:
# Grouping by date, so we can get all the data for all states into one date
# skipnabool, default is True, and all NA/null values are excluded, when computing the result.
data_by_date_US_df = pd.DataFrame(google_df.groupby("dates").mean())
data_by_date_US_df.reset_index(inplace = True)
data_by_date_US_df.head()

In [ ]:
# Reseting the index and saving the cleaned file to csv
data_by_date_US_df.to_csv("google_mob_US.csv")

In [ ]:
# Review previous df for the entire U.S - Noting number of rows
google_df.count

In [ ]:
# Groupby state and date to return the moving average (30 days)
data_us_df = pd.DataFrame(google_df.groupby(['states','dates']).mean())
data_us_df.reset_index(inplace = True)
data_us_df

In [ ]:
# Export the US data csv
data_us_df.to_csv("data_us.csv")

In [ ]:
# start_date = "2020-02-15"
# end_date = "2020-03-01"
# mask = (google_df["dates"] > start_date) & (google_df["dates"] <= end_date)
# cut_date_df = google_df.loc[mask]
# cut_date_df

## Store NYT COVID cases and deaths CSV into DataFrame

In [ ]:
csv_file = "Resources/COVID-states.csv"
covid_data_df = pd.read_csv(csv_file)
covid_data_df.head()

In [ ]:
covid_us = covid_data_df.rename(columns = {"state":'states', "fips": 'fips', "date":'dates', "cases":'cases',
                            "deaths":'deaths'})
covid_us.head()

### Connect to local database

In [ ]:
rds_connection_string = f"{db_user}:{{db_pwd}}@localhost:5432/mobility_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [ ]:
engine.table_names()

### Use pandas to load csv converted DataFrame into database

In [ ]:
google_us.to_sql(name='google_data', con=engine, if_exists='append', index=False)

In [ ]:
covid_us.to_sql(name='covid_data', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the tables

In [ ]:
pd.read_sql_query('select * from google_data', con=engine).head(10)

In [ ]:
pd.read_sql_query('select * from covid_data', con=engine).head(10)